In [1]:
import pandas as pd
from lifelines import CoxPHFitter
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
# Load the data from the CSV file
data_folder = '../data/equity-post-HCT-survival-predictions'

data_dictionary = pd.read_csv(data_folder + '/data_dictionary.csv')
sample_submission = pd.read_csv(data_folder + '/sample_submission.csv')
test = pd.read_csv(data_folder + '/test.csv')
train = pd.read_csv(data_folder + '/train.csv')

In [3]:
# Display the data
data_dictionary

,variable,description,type,values
0,dri_score,Refined disease risk index,Categorical,['Intermediate' 'High' 'N/A - non-malignant in...
1,psych_disturb,Psychiatric disturbance,Categorical,['Yes' 'No' nan 'Not done']
2,cyto_score,Cytogenetic score,Categorical,['Intermediate' 'Favorable' 'Poor' 'TBD' nan '...
3,diabetes,Diabetes,Categorical,['No' 'Yes' nan 'Not done']
4,hla_match_c_high,Recipient / 1st donor allele level (high resol...,Numerical,NaN
5,hla_high_res_8,Recipient / 1st donor allele-level (high resol...,Numerical,NaN
6,tbi_status,TBI,Categorical,"['No TBI' 'TBI + Cy +- Other' 'TBI +- Other, <..."
7,arrhythmia,Arrhythmia,Categorical,['No' nan 'Yes' 'Not done']
8,hla_low_res_6,Recipient / 1st donor antigen-level (low resol...,Numerical,NaN
9,graft_type,Graft type,Categorical,['Peripheral blood' 'Bone marrow']


In [4]:
# Display the testing data
test.head()

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,karnofsky_score,hepatic_mild,tce_div_match,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10
0,28800,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,90.0,No,NaN,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0
1,28801,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0
2,28802,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,90.0,No,Permissive mismatched,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0


In [5]:
# Get numerical and categorical features from data_dictionary
numerical_features = data_dictionary[data_dictionary['type'] == 'Numerical']['variable'].tolist()
categorical_features = data_dictionary[data_dictionary['type'] == 'Categorical']['variable'].tolist()

categorical_features.remove('efs')
numerical_features.remove('efs_time')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), numerical_features),
        ('cat', Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))]), categorical_features)
    ]
)

In [6]:
# Split the data
X = train.drop(columns=['efs', 'efs_time'])
y = train[['efs', 'efs_time']]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Fit the model
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)

In [ ]:
# Get feature names for numerical and categorical features
num_feature_names = numerical_features
cat_feature_names = preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_features)
all_feature_names = num_feature_names + list(cat_feature_names)

# Calculate VIF for each feature
X_train_df = pd.DataFrame(X_train_preprocessed, columns=all_feature_names)
vif_data = pd.DataFrame()
vif_data["feature"] = X_train_df.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_df.values, i) for i in range(len(X_train_df.columns))]

# Filter out features with high VIF
high_vif_features = vif_data[vif_data["VIF"] > 10]["feature"].tolist()
X_train_filtered = X_train_df.drop(columns=high_vif_features)
X_val_filtered = pd.DataFrame(X_val_preprocessed, columns=all_feature_names).drop(columns=high_vif_features)

# Fit the model with filtered features
cph = CoxPHFitter()
cph.fit(pd.concat([X_train_filtered, y_train.reset_index(drop=True)], axis=1), duration_col='efs_time', event_col='efs')

/home/kulac/projects/ML/venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


<lifelines.CoxPHFitter: fitted with 23040 total observations, 10585 right-censored observations>

In [9]:
# Evaluate the model
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 23040 total observations, 10585 right-censored observations>
             duration col = 'efs_time'
                event col = 'efs'
      baseline estimation = breslow
   number of observations = 23040
number of events observed = 12455
   partial log-likelihood = -119878.19
         time fit was run = 2025-01-04 09:05:52 UTC

---
                     coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                            
hla_match_c_high     0.03      1.03      0.01           -0.00            0.06                1.00                1.06
hla_match_dqb1_high  0.03      1.03      0.01            0.01            0.06                1.01                1.06
hla_nmdp_6          -0.04      0.96      0.02           -0.08           -0.01                0.93                0.99
hla_match_c_low     -0.04      0.96      0.01           -0.07           -0.01                0.94                0.99
hla_match_dqb1_low   0.02      1.02      0.01           -0.00            0.05                1.00                1.05
year_hct            -0.13      0.88      0.01           -0.15           -0.12                0.86                0.89
hla_match_a_high    -0.04      0.96      0.01           -0.07           -0.02                0.93                0.98
donor_age            0.02      1.02      0.01            0.01            0.04                1.01                1.04
age_at_hct           0.23      1.26      0.01            0.21            0.25                1.24                1.29
hla_match_a_low     -0.07      0.94      0.01           -0.10           -0.04                0.91                0.96
hla_match_b_high     0.00      1.00      0.01           -0.02            0.03                0.98                1.03
comorbidity_score    0.18      1.20      0.01            0.16            0.20                1.18                1.22
karnofsky_score     -0.09      0.91      0.01           -0.11           -0.08                0.89                0.93
hla_match_drb1_high  0.12      1.12      0.02            0.09            0.15                1.09                1.16

                     cmp to      z      p  -log2(p)
covariate                                          
hla_match_c_high       0.00   1.86   0.06      3.99
hla_match_dqb1_high    0.00   2.66   0.01      7.00
hla_nmdp_6             0.00  -2.23   0.03      5.29
hla_match_c_low        0.00  -2.63   0.01      6.87
hla_match_dqb1_low     0.00   1.78   0.08      3.72
year_hct               0.00 -14.79 <0.005    162.03
hla_match_a_high       0.00  -3.11 <0.005      9.07
donor_age              0.00   2.67   0.01      7.02
age_at_hct             0.00  23.36 <0.005    398.40
hla_match_a_low        0.00  -4.51 <0.005     17.22
hla_match_b_high       0.00   0.34   0.73      0.45
comorbidity_score      0.00  20.73 <0.005    314.84
karnofsky_score        0.00 -10.28 <0.005     79.87
hla_match_drb1_high    0.00   7.42 <0.005     42.93
---
Concordance = 0.61
Partial AIC = 239784.37
log-likelihood ratio test = 1883.14 on 14 df
-log2(p) of ll-ratio test = inf

In [10]:
"""
To evaluate the equitable prediction of transplant survival outcomes,
we use the concordance index (C-index) between a series of event
times and a predicted score across each race group.
 
It represents the global assessment of the model discrimination power:
this is the model's ability to correctly provide a reliable ranking
of the survival times based on the individual risk scores.
 
The concordance index is a value between 0 and 1 where:
 
0.5 is the expected result from random predictions,
1.0 is perfect concordance (with no censoring, otherwise <1.0),
0.0 is perfect anti-concordance (with no censoring, otherwise >0.0)

"""
import pandas as pd
import pandas.api.types
import numpy as np
from lifelines.utils import concordance_index

class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    >>> import pandas as pd
    >>> row_id_column_name = "id"
    >>> y_pred = {'prediction': {0: 1.0, 1: 0.0, 2: 1.0}}
    >>> y_pred = pd.DataFrame(y_pred)
    >>> y_pred.insert(0, row_id_column_name, range(len(y_pred)))
    >>> y_true = { 'efs': {0: 1.0, 1: 0.0, 2: 0.0}, 'efs_time': {0: 25.1234,1: 250.1234,2: 2500.1234}, 'race_group': {0: 'race_group_1', 1: 'race_group_1', 2: 'race_group_1'}}
    >>> y_true = pd.DataFrame(y_true)
    >>> y_true.insert(0, row_id_column_name, range(len(y_true)))
    >>> score(y_true.copy(), y_pred.copy(), row_id_column_name)
    0.75
    """
    
    del solution[row_id_column_name]
    del submission[row_id_column_name]
    
    event_label = 'efs'
    interval_label = 'efs_time'
    prediction_label = 'prediction'
    for col in submission.columns:
        if not pandas.api.types.is_numeric_dtype(submission[col]):
            raise ParticipantVisibleError(f'Submission column {col} must be a number')
    # Merging solution and submission dfs on ID
    merged_df = pd.concat([solution, submission], axis=1)
    merged_df.reset_index(inplace=True)
    merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)
    metric_list = []
    for race in merged_df_race_dict.keys():
        # Retrieving values from y_test based on index
        indices = sorted(merged_df_race_dict[race])
        merged_df_race = merged_df.iloc[indices]
        # Calculate the concordance index
        c_index_race = concordance_index(
                        merged_df_race[interval_label],
                        -merged_df_race[prediction_label],
                        merged_df_race[event_label])
        metric_list.append(c_index_race)
    return float(np.mean(metric_list)-np.sqrt(np.var(metric_list)))

In [11]:
# Convert X_val_preprocessed to DataFrame with appropriate column names
X_val_preprocessed_df = pd.DataFrame(X_val_preprocessed, columns=all_feature_names)

# Predict and evaluate using Stratified Concordance Index
y_pred = cph.predict_partial_hazard(X_val_preprocessed_df)

# Prepare data for scoring
y_pred = pd.DataFrame(y_pred, columns=['prediction'])
y_pred.insert(0, 'id', range(len(y_pred)))
y_val.insert(0, 'id', range(len(y_val)))
y_val = y_val.merge(train[['ID', 'race_group']], left_on='id', right_on='ID', how='left').drop(columns=['ID'])

y_pred

,id,prediction
0,0,1.241952
1,1,0.850505
2,2,1.722844
3,3,1.819903
4,4,1.551260
...,...,...
5755,5755,0.971468
5756,5756,0.719866
5757,5757,0.504759
5758,5758,1.158300


In [12]:
# Calculate the score
score(y_val, y_pred, 'id')

0.6038647151223129

In [13]:
# Preprocess the test data
X_test_preprocessed = preprocessor.transform(test.drop(columns=['ID']))

# Convert X_test_preprocessed to DataFrame with appropriate column names
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, columns=all_feature_names)

# Drop high VIF features
X_test_filtered = X_test_preprocessed_df.drop(columns=high_vif_features)

# Predict using the fitted model
y_test_pred = cph.predict_partial_hazard(X_test_filtered)

# Prepare data for scoring
y_test_pred = pd.DataFrame(y_test_pred, columns=['prediction'])
y_test_pred.insert(0, 'ID', test['ID'])

# Display the predictions
y_test_pred

,ID,prediction
0,28800,0.570139
1,28801,1.609877
2,28802,0.656068
